In [1]:
import numpy as np
import tensorflow as tf

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

In [ ]:
# now let's say we have observed following actions (2 actions for each occupant)
# for occ 1 : we see from ind 0 --> ind 2 and ind 2 to ind 3
# for occ 2 : we see from ind 0 --> ind 3 and ind 2 to ind 1
# for occ 3 : we see from ind 1 --> ind 0 and ind 3 to ind 2

# concerned matrix indices should work as follows
# for occ 1 [0,2,2,3]
# for occ 2 [0,2,3,1]
# for occ 3 [1,3,0,2]
# concerned matrix is then given as follow;
# [1, 3, 3, 4]
# [5, 7, 6, 8]
# [10, 9, 12, 11]

In [17]:
values_numpy = np.array([[1,2,3,4, 13, 14],[5,8,7,6, 15, 16], [12,10,11,9, 17, 18]])
values_tf = tf.constant(values_numpy)
concerned_ind_numpy = np.array([[0,2,2,3],[0,2,3,1],[1,3,0,2]])
concerned_ind = tf.constant(concerned_ind_numpy)

In [12]:
def gather_concerned_utilities(utility_matrix, concerned_ind_mat):
    """
    Gather concerned utility function values for each occupant
    """
    
    splits = tf.split(concerned_ind_mat, num_or_size_splits=concerned_ind_mat.shape[1],axis = 1)
    res_list = []
    for i,s in enumerate(splits):
        print i
        idx = tf.stack([tf.reshape(tf.range(utility_matrix.shape[0]), (-1,1)), tf.to_int32(s)], axis=-1)
        res = tf.gather_nd(utility_matrix, idx)
        res_flatten = tf.reshape(res, [-1])
        res_list.append(res_flatten)
    concerned_mat = tf.transpose(tf.stack(res_list)) 
    return concerned_mat

In [13]:
concerned_mat = gather_concerned_utilities(values, concerned_ind)

0
1
2
3


In [52]:
with tf.Session() as sess:
    e = sess.run(concerned_mat) 
print e

[[ 1  3  3  4]
 [ 5  7  6  8]
 [10  9 12 11]]


In [53]:
U_cur, U_prev = tf.split(concerned_mat, num_or_size_splits=2, axis = 1)

In [54]:
with tf.Session() as sess:
    U_cur = sess.run(U_cur) 
print U_cur

[[ 1  3]
 [ 5  7]
 [10  9]]


In [66]:
a = values_numpy[0][:,None]
b = values_numpy[1][:,None]
a_tf = tf.constant(a)
b_tf = tf.constant(b)

In [67]:
l = []

In [68]:
l.append(a_tf)

In [69]:
l.append(b_tf)

In [72]:
ss = tf.concat(l, 1)

In [79]:
ss = tf.cast(ss, tf.float32)

In [80]:
s_transpose = tf.transpose(ss)

In [81]:
s_transpose

<tf.Tensor 'transpose_2:0' shape=(2, 6) dtype=float32>

In [82]:
mult = tf.matmul(ss, s_transpose)

In [83]:
mult

<tf.Tensor 'MatMul_1:0' shape=(6, 6) dtype=float32>

In [84]:
concerned_mat

<tf.Tensor 'transpose:0' shape=(3, 4) dtype=int64>

In [91]:
flatten_concerned_mat = tf.reshape(concerned_mat, [-1, 1])

In [92]:
flatten_concerned_mat.shape

TensorShape([Dimension(12), Dimension(1)])

In [98]:
with tf.Session() as sess:
    e = sess.run(values) 

In [100]:
e

array([[ 1,  2,  3,  4, 13, 14],
       [ 5,  8,  7,  6, 15, 16],
       [12, 10, 11,  9, 17, 18]])

In [96]:
tf.eye?

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`